In [ ]:
def clinical_preprocessing(clinical_data,
                           data_handling='drop',
                           data_subset=None,
                           encoding=None,
                           view_encoding=None,
                           normalization=None):
    
    
    if data_subset is not None:
        """not including KPS, PsP score or MGMT since too little data points"""
        clinical_data = clinical_data.loc[:,['Gender','Age_at_scan_years','Survival_from_surgery_days','IDH1','GTR_over90percent']]
        
    if data_handling is not None:
        if data_handling == 'drop':
            initial = len(clinical_data)
            clinical_data = clinical_data.dropna(subset=['Survival_from_surgery_days','GTR_over90percent']) #dropna cannot be applied to KPS PsP score or MGMT since this eliminates all 
            print('number of entries deleted:',initial-len(clinical_data))
            
        elif data_handling == 'imputer': 
            numerical_ix = clinical_data.select_dtypes(include=['int64','float64']).columns
            #categorical_ix = X.select_dtypes(include=['object','bool']).columns
            transformer = ColumnTransformer(transformers=[
                ('numerical', SimpleImputer(strategy=numeric_data_handling),numerical_ix), 
                ('categorical', SimpleImputer(strategy='most_frequent'),'GTR_over90percent')], #definitely distorts data #https://medium.com/analytics-vidhya/ways-to-handle-categorical-column-missing-data-its-implementations-15dc4a56893#:~:text=Step%201%3A%20Find%20which%20category,and%20keep%20newly%20imputed%20columns.&text=Advantage%3A%20Simple%20and%20easy%20to%20implement%20for%20categorical%20variables%2Fcolumns.
                remainder='passthrough')
    
            clinical_data = pd.DataFrame(transformer.fit_transform(clinical_data))
            clinical_data.columns = clinical_data.columns
            clinical_data.index = clinical_data.index
            
        
        #else: #create model or randomize #balance dataset ??
            
    
    if normalization is not None:
        if normalization == 'strdscaler':
            scaler = StandardScaler()
            num_d = clinical_data.select_dtypes(exclude=['object','bool'])
            clinical_data[num_d.columns] = scaler.fit_transform(num_d)
            
             
        #if normalization==minmax:
            
    if encoding is not None: 
        if encoding == 'one hot': ##??use column transformer
            ohe = OneHotEncoder(drop=None,handle_unknown='ignore')
            le = LabelEncoder()       
    
            for column in ['Gender','IDH1','GTR_over90percent']:
                category_column = 'Numeric_' + column
                clinical_data[category_column] = le.fit_transform(clinical_data[column])
                d_enc_df = pd.DataFrame(one.fit_transform(clinical_data[[category_column]]).toarray())
                clinical_data = clinical_data.join(d_enc_df)
                
            if view_encoding is not None: 
                print(clinical_data.groupby(column)[category_column].unique())
            
        if encoding == 'label':
            categorical_ix = clinical_data.select_dtypes(include=['object','bool']).columns
            le = LabelEncoder()
            clinical_data[categorical_ix] = clinical_data[categorical_ix].apply(lambda col:le.fit_transform(col))
            clinical_data = clinical_data.join(d_enc_df)
            
    return clinical_data